Resolution modification

In [9]:
from PIL import Image,ImageEnhance
from dotenv import load_dotenv 
import boto3
import os
import numpy as np
import math
import ast
import cv2

In [2]:

dotenv_path = os.path.join("..",'.env')
load_dotenv(dotenv_path)

#Connect to AWS Ressource, Credentials are optionals as we are accessing from the same AWS user
client=boto3.client('rekognition',aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME"))
s3 = boto3.client('s3',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME")
)

#Define bucket name to access
bucket_name = os.getenv("BUCKET_NAME")

#Folder of real classroom photos
classroomFolder=os.path.join("..","ImagesRekognition","RealClassroomPhotos")

In [4]:

def calculate_iou(box1, box2):

    name1,x1, y1, w1, h1 = box1
    name2, x2, y2, w2, h2 = box2
    
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    area_union = area_box1 + area_box2 - area_intersect
    
    iou = area_intersect / area_union
    
    return iou


Load Face gallery and obtain bounding boxes and names only

In [3]:
true=True
faceGalleryArray=[
{"classroom1.jpeg156424":{"filename":"classroom1.jpeg","size":156424,"regions":[{"shape_attributes":{"name":"rect","x":910,"y":348,"width":21,"height":27},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":733,"y":402,"width":29,"height":34},"region_attributes":{"name":"AlejandroCasallas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":344,"y":485,"width":38,"height":44},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":612,"y":337,"width":20,"height":23},"region_attributes":{"name":"AlexanderDominguez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":164,"y":481,"width":35,"height":38},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":567,"y":548,"width":57,"height":58},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":403,"y":582,"width":59,"height":78},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":693,"y":332,"width":20,"height":22},"region_attributes":{"name":"EstebanToro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":15,"y":506,"width":35,"height":40},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1189,"y":330,"width":31,"height":31},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":122,"y":538,"width":38,"height":55},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":440,"y":368,"width":24,"height":26},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":806,"y":460,"width":47,"height":55},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1158,"y":366,"width":26,"height":33},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":399,"width":30,"height":37},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":717,"y":350,"width":22,"height":27},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":547,"y":431,"width":37,"height":44},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":372,"y":427,"width":28,"height":33},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":616,"y":374,"width":30,"height":37},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom2.jpeg152579":{"filename":"classroom2.jpeg","size":152579,"regions":[{"shape_attributes":{"name":"rect","x":910,"y":347,"width":25,"height":28},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":729,"y":394,"width":31,"height":35},"region_attributes":{"name":"AlejandroCasallas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":340,"y":481,"width":37,"height":46},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":617,"y":336,"width":20,"height":25},"region_attributes":{"name":"AlexanderDominguez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":160,"y":481,"width":31,"height":37},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":580,"y":541,"width":55,"height":65},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":428,"y":580,"width":51,"height":76},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":693,"y":333,"width":20,"height":20},"region_attributes":{"name":"EstebanToro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":28,"y":504,"width":29,"height":38},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1185,"y":328,"width":27,"height":32},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":171,"y":538,"width":40,"height":57},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":443,"y":366,"width":23,"height":27},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":809,"y":459,"width":46,"height":53},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1126,"y":375,"width":25,"height":33},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1039,"y":395,"width":25,"height":36},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":720,"y":350,"width":20,"height":30},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":542,"y":428,"width":38,"height":43},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":373,"y":425,"width":27,"height":32},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":613,"y":370,"width":35,"height":39},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom3.jpg155001":{"filename":"classroom3.jpg","size":155001,"regions":[{"shape_attributes":{"name":"rect","x":526,"y":587,"width":59,"height":70},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":605,"y":499,"width":37,"height":47},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":537,"width":43,"height":51},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":452,"width":31,"height":39},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":408,"width":23,"height":32},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1107,"y":430,"width":26,"height":32},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1026,"y":479,"width":31,"height":37},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":356,"y":707,"width":66,"height":80},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":60,"y":790,"width":76,"height":90},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom4.jpg156291":{"filename":"classroom4.jpg","size":156291,"regions":[{"shape_attributes":{"name":"rect","x":552,"y":592,"width":48,"height":62},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":609,"y":500,"width":35,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":770,"y":538,"width":39,"height":51},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":453,"width":28,"height":39},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":480,"width":31,"height":37},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":431,"width":25,"height":33},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":410,"width":25,"height":30},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom5.jpg156524":{"filename":"classroom5.jpg","size":156524,"regions":[{"shape_attributes":{"name":"rect","x":551,"y":592,"width":49,"height":59},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":769,"y":540,"width":42,"height":49},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":607,"y":500,"width":36,"height":45},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":452,"width":29,"height":41},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1020,"y":480,"width":33,"height":39},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":431,"width":26,"height":34},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":410,"width":25,"height":30},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom6.jpg153504":{"filename":"classroom6.jpg","size":153504,"regions":[{"shape_attributes":{"name":"rect","x":59,"y":845,"width":67,"height":74},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":221,"y":721,"width":58,"height":72},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":434,"y":701,"width":50,"height":60},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":584,"y":657,"width":47,"height":47},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":772,"y":616,"width":42,"height":45},"region_attributes":{"name":"EdwinUtria","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":976,"y":539,"width":29,"height":37},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1077,"y":516,"width":26,"height":30},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":957,"y":472,"width":25,"height":28},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":903,"y":501,"width":23,"height":29},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":811,"y":526,"width":21,"height":29},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":704,"y":532,"width":31,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":574,"y":567,"width":40,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":343,"y":557,"width":27,"height":29},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":243,"y":581,"width":30,"height":31},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":290,"y":520,"width":21,"height":26},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":193,"y":553,"width":24,"height":26},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":155,"y":576,"width":31,"height":36},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom7.jpg155011":{"filename":"classroom7.jpg","size":155011,"regions":[{"shape_attributes":{"name":"rect","x":46,"y":849,"width":64,"height":79},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":219,"y":724,"width":58,"height":71},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":430,"y":696,"width":55,"height":65},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":589,"y":661,"width":46,"height":49},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":612,"width":44,"height":54},"region_attributes":{"name":"EdwinUtria","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":969,"y":539,"width":32,"height":39},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1069,"y":518,"width":29,"height":33},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":954,"y":475,"width":25,"height":28},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":898,"y":501,"width":27,"height":29},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":807,"y":527,"width":24,"height":29},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":575,"y":569,"width":38,"height":47},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":706,"y":531,"width":31,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":343,"y":557,"width":27,"height":31},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":290,"y":505,"width":24,"height":29},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":244,"y":579,"width":28,"height":31},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":186,"y":555,"width":24,"height":26},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":134,"y":574,"width":31,"height":38},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom01001.jpg150320":{"filename":"classroom01001.jpg","size":150320,"regions":[{"shape_attributes":{"name":"rect","x":57,"y":806,"width":63,"height":74},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":426,"y":726,"width":44,"height":63},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":568,"y":630,"width":62,"height":69},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":775,"y":555,"width":38,"height":50},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":609,"y":519,"width":41,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":796,"y":465,"width":29,"height":44},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1090,"y":499,"width":27,"height":36},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1158,"y":416,"width":20,"height":32},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1144,"y":436,"width":19,"height":38},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom10110.jpg149420":{"filename":"classroom10110.jpg","size":149420,"regions":[{"shape_attributes":{"name":"rect","x":67,"y":814,"width":53,"height":82},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":623,"y":650,"width":46,"height":55},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":128,"y":590,"width":30,"height":35},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":162,"y":570,"width":26,"height":29},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":223,"y":585,"width":25,"height":30},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":296,"y":541,"width":24,"height":28},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":388,"y":577,"width":25,"height":31},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":576,"y":567,"width":34,"height":51},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":814,"y":523,"width":23,"height":32},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":882,"y":516,"width":23,"height":31},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":932,"y":494,"width":24,"height":30},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1078,"y":520,"width":26,"height":31},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1022,"y":566,"width":28,"height":28},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom28.jpg154093":{"filename":"classroom28.jpg","size":154093,"regions":[{"shape_attributes":{"name":"rect","x":22,"y":550,"width":31,"height":47},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":326,"y":507,"width":28,"height":35},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":136,"y":699,"width":56,"height":66},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":92,"y":592,"width":45,"height":55},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":584,"y":607,"width":50,"height":60},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":515,"y":510,"width":42,"height":48},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1145,"y":404,"width":25,"height":35},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1006,"y":479,"width":33,"height":38},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":798,"y":471,"width":28,"height":35},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":625,"y":438,"width":25,"height":33},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":431,"y":432,"width":23,"height":28},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":901,"y":583,"width":50,"height":47},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}

]



In [51]:
true=True
faceGallery=[
    faceGalleryArray[9]
]
tuples=[]
m=0
for element in faceGallery:
    i=1
    filteredFaceGalleryObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredFaceGalleryObject:
        region=element["shape_attributes"]
        identity=element["region_attributes"]
        tuplesnthPhoto.append((identity["name"],region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1

groundTruthNames=[]
for element in tuples[0]:
    groundTruthNames.append(element[0])


Reduce image's resolution until reaching Rekognition's limit

In [ ]:
selectedCourse="Robotics"
student_photos_path=os.path.join("..","ImagesRekognition",f"Students{selectedCourse}")
camera_frame=os.path.join("..","ImagesRekognition","RealClassroomPhotos","classroom11100.jpg")
camera_photo= Image.open(camera_frame)
cameraWidth,cameraHeight=camera_photo.size

with open(camera_frame, 'rb') as image_file:
    camera_frame_bytes = image_file.read()
results=[]

i=0
for student in os.listdir(student_photos_path):
    if student.split(".")[0] in groundTruthNames:
        if student.split(".")[1]=="jpeg":
            imageQuality=100
        elif student.split(".")[1]=="jpg":
            imageQuality=95
        multiplier=1
        #Define path to individual photo
        student_id_photo=os.path.join("..","imagesRekognition",f"Students{selectedCourse}",student)
        #Obtain image's dimensions in order to know if its size is valid for the minimal rekognition's requirements
        baseImage=Image.open(student_id_photo)
        initialWidth, initialHeight=baseImage.size
        initialBox=True

        while ((initialWidth/multiplier)>80 and (initialHeight/multiplier)>80):
            #Take base image, resize and save it according to a multiplier
            finalWidth=initialWidth/multiplier
            finalHeight=initialHeight/multiplier
            student_id_photo_resized=baseImage.resize((int(finalWidth),int(finalHeight)))
            student_id_photo_resized.save(f'{student}_scaled_{finalHeight}x{finalWidth}.jpg',quality=imageQuality)
            student_id_photo_resized_path=f'{student}_scaled_{finalHeight}x{finalWidth}.jpg'
            size=os.path.getsize(student_id_photo_resized_path)
            TPSimilarityAverage=0
            FPSimilarityAverage=0
            FPCounter=0
            TP=TN=FN=FP=0
            #Convert to bytes the resixed image
            with open(student_id_photo_resized_path, 'rb') as image_file:
                student_id_photo_resized_bytes = image_file.read()

            #Look for the resized students photo in the group photo, reduce threshold to obtain similarity info about all faces detected


            responseRecognition=client.compare_faces(
                        SourceImage={'Bytes': student_id_photo_resized_bytes},
                        TargetImage={'Bytes': camera_frame_bytes},
                        SimilarityThreshold=0
                    )
            
            #Save the corresponding face gallery coordinates in order to compare
            if (initialBox):
                for element in tuples[0]:
                    if element[0]==student.split(".")[0]:
                        trueBoundingBoxWithId=(student,element[1]/cameraWidth,element[2]/cameraHeight,
                                               element[3]/cameraWidth,element[4]/cameraHeight)
                        break

            initialBox=False

            print(f"Results for resolution {finalWidth}x{finalHeight}")

            for face in responseRecognition["FaceMatches"]:
                partialBB=face["Face"]["BoundingBox"]
                IoU=calculate_iou(trueBoundingBoxWithId, ("Name",partialBB["Left"],partialBB["Top"],partialBB["Width"],partialBB["Height"]))
                if (IoU>=0.5):
                    TPSimilarityAverage=TPSimilarityAverage+float(face["Similarity"])
                    if(face["Similarity"]>80):
                        TP=TP+1
                    print(f"{student.split(".")[0]} recognized in {partialBB} with a similarity of {face["Similarity"]}")
                else:
                    FPSimilarityAverage=FPSimilarityAverage+float(face["Similarity"])
                    FPCounter=FPCounter+1
                    if face["Similarity"]>80:
                        FP=FP+1
                        print(f"Error occurred, the model mismatched a face in {partialBB} with {face["Similarity"]}")
                    else:
                        TN=TN+1
            if (TP==0):
                FN=1
            results.append({"Name":student.split(".")[0],"Size": size,"Width": finalWidth, "Height": finalHeight, "TP Similarity": TPSimilarityAverage,
                            "Others Similarity Average": FPSimilarityAverage/FPCounter,"TP":TP,"TN":TN,"FP": FP,"FN":FN})
            multiplier=multiplier*2
    else:
        print(f"{student.split(".")[0]} not in ground truth array ")

    i=i+1


Save partial results

In [19]:
with open(os.path.join("..","Data","resolutionResults.txt"), "r") as file:
    content = file.read()

resolutionControlResults = ast.literal_eval(content)


In [ ]:
biggestResolution=0
leastResolution=99999999
numberOfElements=0

#Obtain number of elements, greatest and lowest resolution value in order to apply Larson's rule
for element in resolutionControlResults:
    for object in element:
        if(object["Size"]>biggestResolution):
            biggestResolution=object["Size"]
        if(object["Size"]<leastResolution):
            leastResolution=object["Size"]
        numberOfElements=numberOfElements+1


numberOfIntervals=math.ceil(1+3.332*np.log10(numberOfElements))
step=math.ceil((biggestResolution-leastResolution)/(numberOfIntervals))
print(f"Biggest {biggestResolution} and least {leastResolution}")
print(f"number of intervals: {numberOfIntervals}")
print(f"step : {step}")



Biggest 4160186 and least 4245
number of intervals: 11
step : 377813


Classify using quartiles and obtain performance indexes

In [12]:
objectGroup = []
for element in resolutionControlResults:
    objectGroup.extend(element)

#Sort elements taking into account the key "Size"
objectGroup.sort(key=lambda obj: obj["Size"])

#Compute quartile size
quartileSize = len(objectGroup) // 4
resultsClassified = [[], [], [], []]

#Distribute objects in quartiles. IF the number isn't divisible by four, the objects remaining or missing are deducted or added to the fourth array
for i, obj in enumerate(objectGroup):
    if i < quartileSize:
        resultsClassified[0].append(obj)
    elif i < 2 * quartileSize:
        resultsClassified[1].append(obj)
    elif i < 3 * quartileSize:
        resultsClassified[2].append(obj)
    else:
        resultsClassified[3].append(obj)



In [ ]:

for element in resultsClassified:
   TPSimilarityTotal=0
   OthersSimilarityTotal=0
   faceCounter=0
   TP=TN=FP=FN=0
   for i,object in enumerate(element):
      TPSimilarityTotal=TPSimilarityTotal+object["TP Similarity"]
      TP=TP+object["TP"]
      TN=TN+object["TN"]
      FP=FP+object["FP"]
      FN=FN+object["FN"]

      faceCounter=faceCounter+object["TP"]+object["TN"]+object["FP"]+object["FN"]-1
      OthersSimilarityTotal=OthersSimilarityTotal+(object["Others Similarity Average"])*(object["TP"]+object["TN"]+object["FP"]+object["FN"]-1)
   SNS=TP/(TP+FN)
   PRC=TP/(TP+FP)
   print(f"Resolution between {element[0]["Size"]/1000}kb and {element[-1]["Size"]/1000}kb ------- Elements in quartile {i+1}")
   print(f"TP similarity average was : {TPSimilarityTotal/(i+1)}")
   print(f"Other faces similarity average was : {OthersSimilarityTotal/faceCounter} ")
   print(f"F1-Score was {2*((SNS*PRC)/(SNS+PRC))}")
   print(f"TP:{TP} TN:{TN} FP: {FP} FN: {FN}")
   print("")

Brigthness control

Brigthness value of an image converting it to black and white array and obtaining the mean of the pixels

In [18]:
def calculate_brightness_simple(image_path):
    img = cv2.imread(image_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    avg = np.mean(hsv[:,:, 2])
    per= (avg/255)*100
    return round(per)



In [ ]:
selectedCourse="Robotics"
student_photos_path=os.path.join("..","ImagesRekognition",f"Students{selectedCourse}")
camera_frame=os.path.join("..","ImagesRekognition","RealClassroomPhotos","classroom11100.jpg")
camera_photo= Image.open(camera_frame)
cameraWidth,cameraHeight=camera_photo.size

with open(camera_frame, 'rb') as image_file:
    camera_frame_bytes = image_file.read()
results=[]

i=0
for student in os.listdir(student_photos_path):
    if student.split(".")[0] in groundTruthNames:
        if student.split(".")[1]=="jpeg":
            imageQuality=100
        elif student.split(".")[1]=="jpg":
            imageQuality=95
        #Define path to individual photo
        student_id_photo=os.path.join("..","imagesRekognition",f"Students{selectedCourse}",student)
        #Obtain image's dimensions in order to know if its size is valid for the minimal rekognition's requirements
        baseImage=Image.open(student_id_photo)
        initialBox=True

        brigthnessValuesArray=[0.1, 0.5, 1, 1.5, 3]
        for brightnessValue in brigthnessValuesArray:
            #Take base image, resize and save it according to a multiplier
            studentIdPhotoBrightnessModified=baseImage
            enhancer=ImageEnhance.Brightness(studentIdPhotoBrightnessModified)
            enhanced_img=enhancer.enhance(brightnessValue)
            enhanced_img.save(rf"modifiedBrightnessImages\{brightnessValue}_{student}")

            studentIdPhotoBrightnessModifiedPath=rf"modifiedBrightnessImages\{brightnessValue}_{student}"
            realBrightness=calculate_brightness_simple(rf"modifiedBrightnessImages\{brightnessValue}_{student}")
            TPSimilarityAverage=0
            FPSimilarityAverage=0
            FPCounter=0
            TP=TN=FN=FP=0
            #Convert to bytes the resixed image
            with open(studentIdPhotoBrightnessModifiedPath, 'rb') as image_file:
                studentIdPhotoBrightnessModifiedPathBytes = image_file.read()
            #Look for the resized students photo in the group photo, reduce threshold to obtain similarity info about all faces detected
            responseRecognition=client.compare_faces(
                        SourceImage={'Bytes': studentIdPhotoBrightnessModifiedPathBytes},
                        TargetImage={'Bytes': camera_frame_bytes},
                        SimilarityThreshold=0
                    )
            
            #Save the corresponding face gallery coordinates in order to compare
            if (initialBox):
                for element in tuples[0]:
                    if element[0]==student.split(".")[0]:
                        trueBoundingBoxWithId=(student,element[1]/cameraWidth,element[2]/cameraHeight,
                                               element[3]/cameraWidth,element[4]/cameraHeight)
                        break

            initialBox=False

            print(f"Results for brightness {brightnessValue}")

            for face in responseRecognition["FaceMatches"]:
                partialBB=face["Face"]["BoundingBox"]
                IoU=calculate_iou(trueBoundingBoxWithId, ("Name",partialBB["Left"],partialBB["Top"],partialBB["Width"],partialBB["Height"]))
                if (IoU>=0.5):
                    TPSimilarityAverage=TPSimilarityAverage+float(face["Similarity"])
                    if(face["Similarity"]>80):
                        TP=TP+1
                    print(f"{student.split(".")[0]} recognized in {partialBB} with a similarity of {face["Similarity"]}")
                else:
                    FPSimilarityAverage=FPSimilarityAverage+float(face["Similarity"])
                    FPCounter=FPCounter+1
                    if face["Similarity"]>80:
                        FP=FP+1
                        print(f"Error occurred, the model mismatched a face in {partialBB} with {face["Similarity"]}")
                    else:
                        TN=TN+1
            if (TP==0):
                FN=1
            results.append({"Name":student.split(".")[0],"Brightness Modifying Factor":brightnessValue, "Real Measured Brightness": realBrightness , "TP Similarity": TPSimilarityAverage,
                            "Others Similarity Average": FPSimilarityAverage/FPCounter,"TP":TP,"TN":TN,"FP": FP,"FN":FN})
    else:
        print(f"{student.split(".")[0]} not in ground truth array ")

    i=i+1


Store data

In [54]:
with open(os.path.join("..","Data","brightnessResults.txt"), "r") as file:
    content = file.read()

brightnessControlResults=ast.literal_eval(content)

In [55]:


TP=[0,0,0,0,0]
TN=[0,0,0,0,0]
FP=[0,0,0,0,0]
FN=[0,0,0,0,0]

TPAverageArray=[0,0,0,0,0]
OthersAverageArray=[0,0,0,0,0]
numberOfOthers=[0,0,0,0,0]
TPCounter=[0,0,0,0,0]

for element in brightnessControlResults:
    counter=0
    for object in element:
        TPAverageArray[counter]=TPAverageArray[counter]+object["TP Similarity"]
        TPCounter[counter]=TPCounter[counter]+1

        TP[counter]=TP[counter]+object["TP"]
        TN[counter]=TN[counter]+object["TN"]
        FP[counter]=FP[counter]+object["FP"]
        FN[counter]=FN[counter]+object["FN"]


        OthersAverageArray[counter]=OthersAverageArray[counter]+object["Others Similarity Average"]*((object["TP"]+object["TN"]+object["FP"]+object["FN"]-1))
        numberOfOthers[counter]=numberOfOthers[counter]+(object["TP"]+object["TN"]+object["FP"]+object["FN"]-1)
        counter=counter+1
        counter = 0 if counter == 5 else counter


for i,result in enumerate(OthersAverageArray):
    SNS=TP[i]/(TP[i]+FN[i])
    PRC=TP[i]/(TP[i]+FP[i])
    print(f" {brigthnessValuesArray[i]} brightness results -- TP Similarity {TPAverageArray[i]/TPCounter[i]} -- Others Similarity Average {(OthersAverageArray[i])/(numberOfOthers[i])} -- F1-Score : {2*((SNS*PRC)/(SNS+PRC))}")



 0.1 brightness results -- TP Similarity 85.71615366233412 -- Others Similarity Average 1.7871081993212596 -- F1-Score : 0.8879310344827586
 0.5 brightness results -- TP Similarity 86.43841340929963 -- Others Similarity Average 1.7454582111755306 -- F1-Score : 0.8831168831168831
 1 brightness results -- TP Similarity 87.02862588010093 -- Others Similarity Average 1.7541803565674727 -- F1-Score : 0.8927038626609441
 1.5 brightness results -- TP Similarity 86.90807755788167 -- Others Similarity Average 1.7262829699174258 -- F1-Score : 0.8879310344827586
 3 brightness results -- TP Similarity 79.85356137734051 -- Others Similarity Average 1.7486512421911433 -- F1-Score : 0.8110599078341014


In [66]:
objectGroup = []
for element in brightnessControlResults:
    objectGroup.extend(element)

#Sort elements taking into account the key "Size"
objectGroup.sort(key=lambda obj: obj["Real Measured Brightness"])

#Compute quartile size
quartileSize = len(objectGroup) // 5
resultsClassifiedBrightness = [[], [], [], [], []]

#Distribute objects in quartiles. IF the number isn't divisible by four, the objects remaining or missing are deducted or added to the fourth array
for i, obj in enumerate(objectGroup):
    if i < quartileSize:
        resultsClassifiedBrightness[0].append(obj)
    elif i < 2 * quartileSize:
        resultsClassifiedBrightness[1].append(obj)
    elif i < 3 * quartileSize:
        resultsClassifiedBrightness[2].append(obj)
    elif i < 4 * quartileSize:
        resultsClassifiedBrightness[3].append(obj)
    else:
        resultsClassifiedBrightness[4].append(obj)



In [ ]:
for element in resultsClassifiedBrightness:
   TPSimilarityTotal=0
   OthersSimilarityTotal=0
   faceCounter=0
   TP=TN=FP=FN=0
   for i,object in enumerate(element):
      TPSimilarityTotal=TPSimilarityTotal+object["TP Similarity"]
      TP=TP+object["TP"]
      TN=TN+object["TN"]
      FP=FP+object["FP"]
      FN=FN+object["FN"]

      faceCounter=faceCounter+object["TP"]+object["TN"]+object["FP"]+object["FN"]-1
      OthersSimilarityTotal=OthersSimilarityTotal+(object["Others Similarity Average"])*(object["TP"]+object["TN"]+object["FP"]+object["FN"]-1)
   SNS=TP/(TP+FN)
   PRC=TP/(TP+FP)
   print(f"Brightness between {element[0]["Real Measured Brightness"]} and {element[-1]["Real Measured Brightness"]} ------- Elements in quartile {i+1}")
   print(f"TP similarity average was : {TPSimilarityTotal/(i+1)}")
   print(f"Other faces similarity average was : {OthersSimilarityTotal/faceCounter} ")
   print(f"F1-Score was {2*((SNS*PRC)/(SNS+PRC))}")
   print(f"TP:{TP} TN:{TN} FP: {FP} FN: {FN}")
   print("")

In [ ]:

roboticsFolderPath=os.path.join("..",'ImagesRekognition',"StudentsRobotics")

i=0
brightness_counter=0
for image in os.listdir(roboticsFolderPath):
    image_path=os.path.join(roboticsFolderPath,image)
    brightness=calculate_brightness_simple(image_path)
    brightness_counter=brightness_counter+brightness
    i=i+1
    print(f"Initial brightness of {image.split(".")[0]} is {brightness}")

    imageOpened=Image.open(image_path)
    enhancer=ImageEnhance.Brightness(imageOpened)
    enhanced_img=enhancer.enhance(0.05)
    enhanced_img.save(rf"modifiedBrightnessImages\{image}")
    newBrightness=calculate_brightness_simple(rf"modifiedBrightnessImages\{image}")
    print(f"Final brightness of {image.split(".")[0]} is {newBrightness}")


print(f"Average brightness is {float(brightness_counter/i)}")


